## Exercise #1: Data Comparison and Reporting Missing Data

In [ ]:
import pandas as pd

internal_df = pd.read_excel('../data/Exercise #1A.xlsx')
acacia_df = pd.read_excel('../data/Exercise #1B.xlsx')

In [ ]:
print("Internal Data Columns:", internal_df.columns)
print("Acacia Data Columns:", acacia_df.columns)

In [ ]:
filtered_internal_df = internal_df[
    (internal_df['Case Status'] == 'Open') &
    (internal_df['Subpractice'].isin(['LTFC', 'URM'])) &
    (internal_df['Reported Date'].dt.month == 3) &
    (internal_df['Reported Date'].dt.year == 2023)
]

filtered_acacia_df = acacia_df[
    (acacia_df['Case Status'] == 'Open') &
    (acacia_df['Subpractice'].isin(['LTFC', 'URM'])) &
    (acacia_df['Reported Date'].dt.month == 3) &
    (acacia_df['Reported Date'].dt.year == 2023)
]

In [ ]:
missing_df = pd.merge(filtered_internal_df, filtered_acacia_df, on='Case #', how='outer', indicator=True, suffixes=('_internal', '_acacia'))
missing_df = missing_df[missing_df['_merge'] == 'left_only']


missing_df = missing_df.drop(columns=['_merge'])
missing_df

In [ ]:
comparison_df = pd.merge(filtered_internal_df, filtered_acacia_df, on='Case #', suffixes=('_internal', '_acacia'))
comparison_df

In [ ]:
internal_columns = comparison_df.filter(like='_internal').columns
acacia_columns = comparison_df.filter(like='_acacia').columns

mismatch_df = pd.DataFrame(index=comparison_df.index)
mismatch_df['Case #'] = comparison_df['Case #']

for internal_col in internal_columns:
    acacia_col = internal_col.replace('_internal', '_acacia')
    if acacia_col in acacia_columns:
        mismatch_col = f'{internal_col.replace("_internal", "")}_mismatch'
        
        internal_values = comparison_df[internal_col]
        acacia_values = comparison_df[acacia_col]
        
        # Check if both values are NaN or if they are different
        mismatch_df[mismatch_col] = (internal_values != acacia_values) & (~pd.isna(internal_values) | ~pd.isna(acacia_values))
        
        # Add columns for both internal and acacia values
        mismatch_df[internal_col] = comparison_df[internal_col]
        mismatch_df[acacia_col] = comparison_df[acacia_col]

# Combine mismatch information with the original DataFrame
full_comparison_df = pd.concat([comparison_df[['Case #']], mismatch_df], axis=1)


# Extract rows with any mismatches
mismatch_rows = full_comparison_df.filter(like='_mismatch').any(axis=1)
differences_df = full_comparison_df[mismatch_rows].copy()
differences_df

In [ ]:
from openpyxl.styles import PatternFill

# Create a copy of the DataFrame for Excel output
differences_output_df = differences_df.copy()

# Remove '_mismatch' columns from the copy
columns_to_keep = [col for col in differences_output_df.columns if '_mismatch' not in col]
differences_output_df = differences_output_df[columns_to_keep]

# Remove duplicate columns
differences_output_df = differences_output_df.loc[:, ~differences_output_df.columns.duplicated()]

# Filter columns that end with '_internal' and include 'Case #' column
internal_columns = [col for col in missing_df.columns if col.endswith('_internal')]
internal_columns.append('Case #')  # Ensure 'Case #' is included

# Create a new DataFrame with only the relevant columns
missing_output_df = missing_df[internal_columns]

# Rename columns by removing the '_internal' postfix
missing_output_df.columns = [col.replace('_internal', '') if col.endswith('_internal') else col for col in missing_output_df.columns]
missing_output_df = missing_output_df[['Case #'] + [col for col in missing_output_df.columns if col != 'Case #']]

# Create an Excel writer
output_file = '../output/acacia-data-discrepancies.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    
    missing_output_df.to_excel(writer, sheet_name='Missing', index=False)
    # Convert the filtered DataFrame to an Excel sheet
    differences_output_df.to_excel(writer, index=False, sheet_name='Differences')
    
    # Access the Excel workbook and sheet
    workbook = writer.book
    sheet = writer.sheets['Differences']
    
    # Define a fill color for mismatches (e.g., yellow)
    highlight_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
    
    # Apply color coding to mismatched cells based on the original `differences_df`
    for row in range(2, len(differences_df) + 2):  # Start from row 2 (row 1 is the header)
        for col_idx, column in enumerate(differences_df.columns, 1):
            if '_mismatch' in column:
                if differences_df.iloc[row - 2][column]:  # Check if there's a mismatch
                    # Color the internal cell
                    internal_col = f'{column.replace("_mismatch", "_internal")}'
                    acacia_col = f'{column.replace("_mismatch", "_acacia")}'
                    internal_col_idx = differences_output_df.columns.get_loc(internal_col) + 1
                    acacia_col_idx = differences_output_df.columns.get_loc(acacia_col) + 1
                    
                    if internal_col in differences_df.columns:
                        sheet.cell(row=row, column=internal_col_idx).fill = highlight_fill
                    if acacia_col in differences_df.columns:
                        sheet.cell(row=row, column=acacia_col_idx).fill = highlight_fill

print(f"Color-coded Excel file has been saved to {output_file}")

In [ ]:

email_template = '''
Subject: Discrepancies Identified in Acacia Database

Dear Holly,

After a thorough review of the data from Acacia’s database, I have identified several discrepancies when compared to our internal dataset. Specifically, there are {missing_count} records that are absent from Acacia’s database, and {mismatch_count} records with data inconsistencies.

For your reference, I have attached an Excel file titled "acacia-data-discrepancies.xlsx." This file contains two sheets: "Missing," which details the absent records, and "Differences," which highlights the data inconsistencies.

Please let me know if you require any further information or if you would like the data in a different format for ease of use.

Best regards,
Marvin Kassabian
'''

# Fill in the missing and mismatching counts
email_text = email_template.format(missing_count=missing_df.shape[0], mismatch_count=differences_df.shape[0])
print(email_text)
